In [1]:
from preprocess import preprocess_data
import pandas as pd
import numpy as np
import pickle
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
 # Splitting to Train, Test, Validation and Loading data - was run once and test and validation files were used for all files
data1 = pd.read_csv("data.csv")
train_data, temp_data = train_test_split(data1, test_size=0.3, random_state=42)
validation_data, test_data = train_test_split(temp_data, test_size=2/3, random_state=42)
train_data.to_csv("train_data.csv", index=False)
validation_data.to_csv("validation_data.csv", index=False)
test_data.to_csv("test_data.csv", index=False)


data = pd.read_csv("train_data.csv")
test_data = pd.read_csv("test_data.csv")
validation_data = pd.read_csv("validation_data.csv")


In [3]:
data,test_data,validation_data = preprocess_data(data,test_data,validation_data,True)

Starting preprocessing
Converting Julian Time
Adding 4th of July feature
Adding season feature
Handling nulls
Converting dates to hours
Adding hours to containment feature
Adding is_day feature
Adding days_till_cont feature
Create bins for discovered hour
Add holiday feature
KNN classification starting...
Creating dummies for data
FIRE_SIZE_CLASS
Season
discovery_month
classified_as
Creating dummies for test_data
FIRE_SIZE_CLASS
Season
discovery_month
classified_as
Creating dummies for validation_data
FIRE_SIZE_CLASS
Season
discovery_month
classified_as
Removing features for model


#  X & Y Splitting

In [4]:
data.columns

Index(['STAT_CAUSE_DESCR', 'LATITUDE', 'LONGITUDE',
       'is_week_before_or_after_july_4', 'HOURS_TO_CONTAIN', 'IS_DAY',
       'DISCOVERY_HOUR_BIN_10-16', 'DISCOVERY_HOUR_BIN_16-22',
       'DISCOVERY_HOUR_BIN_22-4', 'DISCOVERY_HOUR_BIN_4-10', 'is_weekend',
       'is_holiday', 'FIRE_SIZE_CLASS_A', 'FIRE_SIZE_CLASS_B',
       'FIRE_SIZE_CLASS_C', 'FIRE_SIZE_CLASS_D', 'FIRE_SIZE_CLASS_E',
       'FIRE_SIZE_CLASS_F', 'FIRE_SIZE_CLASS_G', 'Season_Fall',
       'Season_Spring', 'Season_Summer', 'Season_Winter', 'discovery_month_1',
       'discovery_month_2', 'discovery_month_3', 'discovery_month_4',
       'discovery_month_5', 'discovery_month_6', 'discovery_month_7',
       'discovery_month_8', 'discovery_month_9', 'discovery_month_10',
       'discovery_month_11', 'discovery_month_12', 'classified_as_Arson',
       'classified_as_Campfire', 'classified_as_Children',
       'classified_as_Debris Burning', 'classified_as_Equipment Use',
       'classified_as_Fireworks', 'classified_as_

In [5]:
X_train = data.drop(columns = ['STAT_CAUSE_DESCR'])
y_train = data['STAT_CAUSE_DESCR']
X_test = test_data.drop(columns = ['STAT_CAUSE_DESCR'])
y_test = test_data['STAT_CAUSE_DESCR']


# Encoding y to numeric

In [6]:
le = LabelEncoder()
encoded_y_train = le.fit_transform(y_train)
encoded_y_test = le.fit_transform(y_test)


# Random Forest, Grid Search CV 

In [7]:
# Define your model
model = RandomForestClassifier(random_state=10)

# Define the parameter grid for grid search
params_grid = { 
    'n_estimators': [30,40],
    'min_samples_leaf': [16,25,50],
    'min_samples_split': [250,80,120]
}


# Define the scorer as ROC AUC score
scorer = make_scorer(roc_auc_score, needs_proba=True, multi_class='ovr')
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=params_grid, cv=5,scoring=scorer)
grid.fit(X_train, encoded_y_train)

best_model = grid.best_estimator_

# Predict probabilities on the filtered test data
predictions_proba = best_model.predict_proba(X_test)

# Calculate AUC-ROC
auc_roc = roc_auc_score(encoded_y_test, predictions_proba,multi_class='ovr')
print("AUC-ROC:", auc_roc)
print("best params:")
print(grid.best_params_)


AUC-ROC: 0.7853024205728061
best params:
{'min_samples_leaf': 16, 'min_samples_split': 120, 'n_estimators': 40}


# Logistic Regression

In [8]:
# Define the Logistic Regression classifier
logistic_model = LogisticRegression(multi_class='multinomial')

# Train the model
logistic_model.fit(X_train, y_train)

# Make predictions
y_pred_proba = logistic_model.predict_proba(X_test)

# Calculate AUC-ROC score
auc_roc_score = roc_auc_score(y_test, y_pred_proba, multi_class='ovr')
print(str(auc_roc_score))

C:\Users\Avraham\.conda\envs\competitive\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6516877736682029


# Naive bayessian 

In [9]:
nb_classifier = GaussianNB()

# Train the classifier
nb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_proba = nb_classifier.predict_proba(X_test)

# Calculate AUC-ROC score
auc_roc_score = roc_auc_score(y_test, y_pred_proba, multi_class='ovr')

print("AUC-ROC Score:", auc_roc_score)

AUC-ROC Score: 0.6996371824041304


# Neural Network -  was not successful

In [10]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split
# # Convert DataFrame to NumPy arrays
# X_train_np = X_train.values
# X_test_np = X_test.values

# # Encode categorical labels
# label_encoder = LabelEncoder()
# y_train_encoded = label_encoder.fit_transform(y_train)
# y_test_encoded = label_encoder.transform(y_test)

# # Convert data to PyTorch tensors
# X_train_tensor = torch.tensor(X_train_np, dtype=torch.float32)
# y_train_tensor = torch.tensor(y_train_encoded, dtype=torch.long)
# X_test_tensor = torch.tensor(X_test_np, dtype=torch.float32)
# y_test_tensor = torch.tensor(y_test_encoded, dtype=torch.long)

# # Define neural network architecture
# class Classifier(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(Classifier, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden_size)
#         self.fc2 = nn.Linear(hidden_size, output_size)

#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = self.fc2(x)
#         return torch.log_softmax(x, dim=1)

# # Instantiate the model
# input_size = X_train_np.shape[1]  # Number of features
# hidden_size = 64
# output_size = 12  
# model = Classifier(input_size, hidden_size, output_size)

# def roc_auc_loss(y_true, y_pred):
#     try:
#         auc = roc_auc_score(y_true.detach().cpu().numpy(), y_pred.detach().cpu().numpy(), average='macro', multi_class='ovo')
#         return torch.tensor(1 - auc, dtype=torch.float32, requires_grad=True)  # Return 1 - ROC AUC as the loss
#     except ValueError:
#         return torch.tensor(0.5, dtype=torch.float32, requires_grad=True)  # Return 0.5 if AUC cannot be computed

# # Define optimizer
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Train the model
# num_epochs = 10
# for epoch in range(num_epochs):
#     model.train()
#     optimizer.zero_grad()
#     outputs = model(X_train_tensor)
#     loss = roc_auc_loss(y_train_tensor, torch.exp(outputs))  # Use torch.exp(outputs) to convert log probabilities to probabilities
#     loss.backward()
#     optimizer.step()

# # Evaluate the model
# model.eval()
# with torch.no_grad():
#     outputs = model(X_test_tensor)
#     _, predicted = torch.max(outputs, 1)
#     # Use predicted probabilities instead of predicted classes for ROC AUC calculation
#     roc_auc = roc_auc_score(y_test_encoded, torch.exp(outputs).cpu().numpy(), average='macro', multi_class='ovr')

# print(f'Test ROC AUC: {roc_auc:.3f}')


# XGBoosting

In [11]:
# Define the XGBoost classifier
xgb_model = XGBClassifier()

# Define the parameters grid for grid search
param_grid = {
    'n_estimators': [40, 30],
    'max_depth': [5,10],
    'learning_rate': [0.1,0.3],
    'subsample': [0.1,0.3],
    'colsample_bytree': [0.1,0.3]
}
# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='roc_auc', verbose=3)
grid_search.fit(X_train, encoded_y_train)

# # Get the best parameters and the best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# # Train the model with the best parameters
best_xgb_model = XGBClassifier(**best_params)
best_xgb_model.fit(X_train, encoded_y_train)

# # Make predictions
y_pred_proba = best_xgb_model.predict_proba(X_test)

# Calculate AUC-ROC score using one-vs-rest (ovr) strategy
auc_roc_score = roc_auc_score(encoded_y_test, y_pred_proba, multi_class='ovr')
print("AUC-ROC Score:", auc_roc_score)
print("best parameters:")
print(grid_search.best_params_)


Fitting 3 folds for each of 32 candidates, totalling 96 fits


C:\Users\Avraham\.conda\envs\competitive\lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Avraham\.conda\envs\competitive\lib\site-packages\sklearn\model_selection\_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "C:\Users\Avraham\.conda\envs\competitive\lib\site-packages\sklearn\metrics\_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "C:\Users\Avraham\.conda\envs\competitive\lib\site-packages\sklearn\metrics\_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "C:\Users\Avraham\.conda\envs\competitive\lib\site-packages\sklearn\utils\_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)


[CV 1/3] END colsample_bytree=0.1, learning_rate=0.1, max_depth=5, n_estimators=40, subsample=0.1;, score=nan total time=   5.7s


C:\Users\Avraham\.conda\envs\competitive\lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Avraham\.conda\envs\competitive\lib\site-packages\sklearn\model_selection\_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "C:\Users\Avraham\.conda\envs\competitive\lib\site-packages\sklearn\metrics\_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "C:\Users\Avraham\.conda\envs\competitive\lib\site-packages\sklearn\metrics\_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "C:\Users\Avraham\.conda\envs\competitive\lib\site-packages\sklearn\utils\_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)


[CV 2/3] END colsample_bytree=0.1, learning_rate=0.1, max_depth=5, n_estimators=40, subsample=0.1;, score=nan total time=   5.6s


...

[CV 1/3] END colsample_bytree=0.3, learning_rate=0.3, max_depth=10, n_estimators=30, subsample=0.3;, score=nan total time=   5.2s


C:\Users\Avraham\.conda\envs\competitive\lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Avraham\.conda\envs\competitive\lib\site-packages\sklearn\model_selection\_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "C:\Users\Avraham\.conda\envs\competitive\lib\site-packages\sklearn\metrics\_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "C:\Users\Avraham\.conda\envs\competitive\lib\site-packages\sklearn\metrics\_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "C:\Users\Avraham\.conda\envs\competitive\lib\site-packages\sklearn\utils\_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)


[CV 2/3] END colsample_bytree=0.3, learning_rate=0.3, max_depth=10, n_estimators=30, subsample=0.3;, score=nan total time=   5.6s


C:\Users\Avraham\.conda\envs\competitive\lib\site-packages\sklearn\model_selection\_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Avraham\.conda\envs\competitive\lib\site-packages\sklearn\model_selection\_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "C:\Users\Avraham\.conda\envs\competitive\lib\site-packages\sklearn\metrics\_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "C:\Users\Avraham\.conda\envs\competitive\lib\site-packages\sklearn\metrics\_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "C:\Users\Avraham\.conda\envs\competitive\lib\site-packages\sklearn\utils\_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)


[CV 3/3] END colsample_bytree=0.3, learning_rate=0.3, max_depth=10, n_estimators=30, subsample=0.3;, score=nan total time=   5.6s
AUC-ROC Score: 0.7558887903211916
best parameters:
{'colsample_bytree': 0.1, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 40, 'subsample': 0.1}


In [12]:
print("AUC-ROC Score:", auc_roc_score)
print("best parameters:")
print(grid_search.best_params_)

AUC-ROC Score: 0.7558887903211916
best parameters:
{'colsample_bytree': 0.1, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 40, 'subsample': 0.1}
